In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import warnings

sns.set(font_scale=1.3, 
        font='Calibri', 
        style="whitegrid")

## Carregando os dados

### Dados sobre as questões feitas pelos usuários

In [46]:
df_questions = pd.read_json('data/BASE B/questions.json')
df_questions['QuestionDate'] = pd.to_datetime(df_questions['QuestionDate'])
df_questions.head()

,StudentId,QuestionDate,QuestionSnippet,StudentClient
0,12970655,2013-09-04 14:09:25.963,O que é mais importante para um projeto: escop...,NaN
1,12970655,2013-09-04 15:36:51.083,Você pode ter um negócio/produto bem sucedido ...,NaN
2,12970655,2013-10-30 19:41:43.137,Custos com consultoria para planejar um projet...,NaN
3,12970655,2013-10-30 19:19:15.517,CAPEX: investimento em consultoria entra como ...,NaN
4,12970655,2013-10-30 19:38:06.420,Custos com consultoria para planejar um projet...,NaN


In [47]:
df_questions.groupby(['StudentId'])['QuestionSnippet'].count().describe()

count    2240.000000
mean        1.743750
std         3.303801
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        78.000000
Name: QuestionSnippet, dtype: float64

In [3]:
df_questions.groupby(['StudentId'])['QuestionSnippet'].count().sort_values(ascending=False)

StudentId
36619758    78
52684890    76
53791397    59
2449655     45
7308309     24
            ..
62490564     1
62514900     1
62543464     1
62544540     1
49918755     1
Name: QuestionSnippet, Length: 2240, dtype: int64

In [16]:
pd.set_option('display.max_colwidth', 100)
df_questions.query('StudentId == 36619758')['QuestionSnippet']

3202              tenho prova unopar Raciocínio lógico e matemático chama 32988499570
3203                    prova unopar Raciocínio lógico e matemático chama 32988499570
3204                                                         Prova unopar 32988499570
3205                                                    prova unopar wsp 32 988499570
3206          Raciocínio logico e matemático. prova da unopar  chama wsp 88 8136 2988
                                            ...                                      
3275    unopar provas formação integral em saúde hoje tenho prova chama (88)8136-2988
3276      tenho prova formação integral em saúde hoje tenho prova chama (88)8136-2988
3277                      ciência humana socais  hoje tenho prova chama (88)8136-2988
3278                                                        prova unopar 43 9107-7084
3279                              alunos interessado provas unopar chama 43 9107-7084
Name: QuestionSnippet, Length: 78, dtype: object

### Dados sobre as respostas dadas pelos usuários
Fazendo o parsing das respostas que vieram com elementos de HTML.

In [6]:
df_answers = pd.read_json('data/BASE B/answers.json')
df_answers['AnswerDate'] = pd.to_datetime(df_answers['AnswerDate'])

# Supress warnings to run the BeautifulSoup to ignore URLs in the answers
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
original_answers = df_answers['AnswerSnippet'].copy() # Creating a copy to show some stuff later
df_answers['AnswerSnippet'] = df_answers['AnswerSnippet'].map(lambda x: BeautifulSoup(x).text.replace("\n", ""))
warnings.resetwarnings()

df_answers.head()

,StudentId,AnswerDate,AnswerSnippet,StudentClient
0,12970655,2013-08-30 12:27:57.190,Vou tentar simplificar ainda mais...<str,NaN
1,12970655,2013-09-04 14:03:18.240,O centil divide algo em 100 partes. Cada centil é 1 parte das 100 partes.,NaN
2,12970655,2013-09-05 13:20:09.513,Ou será que o problema que esses produtos solucionam é o TÉDIO? =),NaN
3,12970655,2013-09-04 19:01:56.357,Excelente pergunta! Abaixo estão as minhas lições em ordem de importân,NaN
4,12970655,2013-11-12 00:06:05.200,Vamos supor que você se compromete a comprar um imóvel de R$ 500.000 em 50 parcelas,NaN


#### Contagem de respostas por usuário

In [7]:
df_answers.groupby(['StudentId'])['AnswerSnippet'].count().sort_values(ascending=False)

StudentId
70700878    124
52374651     67
53791397     62
78378179     56
60827919     50
           ... 
59300762      1
59230582      1
59133236      1
59051854      1
3546          1
Name: AnswerSnippet, Length: 3068, dtype: int64

#### As 124 respostas me chamaram a atenção e esse aluno parece gostar bastante de postar vários links, inclusives externos ao PD
Não sei se há algum tipo de filtro a esse tipo de conteúdo. Talvez esconder URLs externas ao domínio do PD ou, pelo menos, avisar ao usuário que ele está saindo do domínio e que a plataforma não se responsabiliza a partir desse ponto.

In [8]:
pd.set_option('display.max_colwidth', 100)
df_answers.query('StudentId == 70700878')['AnswerSnippet']

650                                                                        para quem se interessa e
651      aqui neste link tem mais de 300 livros para abaixar que vão te ajudar a tirar suas duvidas
652    Neste link  https://mega.co.nz/#F!Ns4UkDCJ!Ep49t57YcQMIeoiIA6rdpQ tem genes VIII e genes XI 
653            neste link tem alguns livros de patologia que talvez possa te ajudar LINK https://me
654                              neste link tem alguns livros de Hemato para vc abaixar   <a href="
                                                   ...                                             
769              neste link vc vai encontrar livros que vão te ajudar<a href="../../arquivo/1726254
770              neste link vc vai encontrar livros que vão te ajudar<a href="../../arquivo/1726254
771            neste link vc vai encontrar <a href="../../arquivo/17262543/sao-mais-350-livros-com-
772              neste link vc vai encontrar livros que vão te ajudar<a href="../../arquivo/1726254


#### Tamanho das respostas
Os 100 caracteres são o limite para extrair o snippet, porém há respostas muito curtas que muito provavelmente não respondem à pergunta do usuário. Inclusive, algumas respostas são formadas apenas por caracteres especiais.

In [20]:
df_answers[df_answers['AnswerSnippet'].str.match(r'^[^a-zA-Z0-9]+$')]

,StudentId,AnswerDate,AnswerSnippet,StudentClient
82,79626956,2017-11-29 10:42:54.641832,:),Website
153,20863702,2015-08-26 17:12:22.370000,=\,Website
217,28332695,2018-05-17 23:03:05.048545,..,Website
218,28332695,2018-05-17 23:03:10.142826,...,Website
219,28332695,2018-05-17 23:02:52.231273,..,Website
...,...,...,...,...
7408,10269728,2017-11-20 12:23:52.569639,.,Website
7446,61771473,2017-11-21 10:25:06.162337,...,Website
7479,50149742,2018-04-18 21:20:45.305708,?,Website
7510,56220010,2018-05-15 13:31:32.627070,?,Android | 7.0 | sdk 24


### Dados sobre as matérias que cada usuário segue

In [3]:
df_subjects = pd.read_json('data/BASE B/subjects.json')
df_subjects['FollowDate'] = pd.to_datetime(df_subjects['FollowDate'])

df_subjects['year'] = df_subjects.FollowDate.dt.year
df_subjects['month'] = df_subjects.FollowDate.dt.month
df_subjects['year-month'] = df_subjects['year'].astype(str) + '-' + df_subjects['month'].astype(str)

df_subjects.sort_values('FollowDate', inplace=True)
df_subjects.head()

,StudentId,SubjectName,FollowDate,year,month,year-month
63571,87321417,Administração Financeira e Orçamentária I,2015-08-31 18:28:04,2015,8,2015-8
63572,87321417,Administração Financeira e Orçamentária II,2015-08-31 18:28:04,2015,8,2015-8
63569,87321417,Administração Financeira e Orçamentária,2015-08-31 18:28:04,2015,8,2015-8
42314,11356579,Eletrônica de Potência,2015-08-31 18:28:50,2015,8,2015-8
42317,11356579,Geração e Distribuição de Vapor,2015-08-31 18:28:50,2015,8,2015-8


### Quantidade de momentos diferentes em que um usuário se registrou em disciplinas
O primeiro momento é na tela de registro. Caso o usuário não tenha se registrado em mais nenhuma disciplina depois do registro, o valor da agregação será 1, por exemplo.

In [42]:
df_subjects.groupby('StudentId')['FollowDate'].nunique().describe()

count    57229.000000
mean         1.753447
std          3.074462
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        263.000000
Name: FollowDate, dtype: float64

In [54]:
df_subjects.groupby('StudentId')['FollowDate'].nunique().quantile(.70)

1.0

In [41]:
df_subjects.groupby('StudentId')['FollowDate'].nunique().quantile(.85)

2.0

In [19]:
df_subjects.groupby('StudentId')['SubjectName'].count().sort_values(ascending=False).head()

StudentId
41990843    283
96396664    219
63414380    218
37446846    200
50400152    190
Name: SubjectName, dtype: int64

In [29]:
df_subjects[df_subjects.StudentId == 41990843]

,StudentId,SubjectName,FollowDate,year,month,year-month
94342,41990843,Direito Constitucional II,2015-11-15 08:03:53,2015,11,2015-11
94320,41990843,Direito Processual Civil I,2015-11-15 08:03:53,2015,11,2015-11
94312,41990843,Direito Constitucional,2015-11-15 08:03:53,2015,11,2015-11
94340,41990843,Direito Penal I,2015-11-15 08:03:53,2015,11,2015-11
94339,41990843,Direito Constitucional I,2015-11-15 08:03:53,2015,11,2015-11
...,...,...,...,...,...,...
94509,41990843,Teoria Geral do Crime,2016-09-21 09:01:03,2016,9,2016-9
94462,41990843,Teoria Geral do Direito Empresarial,2016-09-21 09:01:04,2016,9,2016-9
94412,41990843,Desenvolvimento Econômico,2016-09-21 09:01:05,2016,9,2016-9
94478,41990843,Teoria da Argumentação Jurídica,2016-09-21 09:01:06,2016,9,2016-9


In [18]:
df_subjects[df_subjects.StudentId == 96396664]

,StudentId,SubjectName,FollowDate,year,month,year-month
24253,96396664,Orientações para A Prática Profissional,2015-09-05 18:22:25.000000,2015,9,2015-9
24157,96396664,Ética Profissional,2015-09-05 18:22:25.000000,2015,9,2015-9
24300,96396664,Serviço Social e Processos de Trabalho I,2015-09-05 18:22:25.000000,2015,9,2015-9
24252,96396664,Legislação Social I,2015-09-05 18:22:25.000000,2015,9,2015-9
24271,96396664,Formação Sócio Econômica e Política da Socieda...,2015-09-05 18:22:25.000000,2015,9,2015-9
...,...,...,...,...,...,...
24359,96396664,Enap,2018-03-14 16:30:15.062138,2018,3,2018-3
24214,96396664,Desigualdade e Pobreza,2018-03-14 16:30:15.174495,2018,3,2018-3
24343,96396664,Serviços Social Relatório de Estágio,2018-04-03 11:21:15.427205,2018,4,2018-4
24361,96396664,Simulado do Enade,2018-04-17 22:00:56.534668,2018,4,2018-4
